In [ ]:
# define matplotlibplotting backend
# %matplotlib -l shows all available backends
%matplotlib qt

In [ ]:
import os
import numpy as np
import string

from lib.parameters import Parameters
from lib.plot_builder import PlotBuilder
from lib.h5_reader import H5Reader
from lib.plain_reader import PlainReader

In [ ]:
##  configuration options
config_file = '../parameters.xml'

cfg = Parameters(config_file)

# update shape to get custom shaped images
# useful to get just part of frame
# or get frames, which has smaller shape than full frame
shape=[0,0,cfg.number_r_grid, cfg.number_z_grid]

timestamp=1e-10
radius=0.01

ylim=None
show_grid=False
use_cache=False
cmap='terrain'

# color limits (WARNING: clim_estimation may works incorrectly)
clim_density = [0, 2e17]
specie = 'electrons'

image_interpolation = 'nearest'

autoselect = True

x_axis_label = r'$\mathit{R (m)}$'
y_axis_label = r'$\mathit{\rho (m^{-3})}$'
cbar_axis_label = r'$\mathit{\rho (m^{-3})}$'

t_plot_name = r'$\mathbf{%s\enspace Density\enspace (\rho)}$' % specie.capitalize()

In [ ]:
# define reader (plain reader used)
if not cfg.use_hdf5:
    reader = PlainReader(path = cfg.data_path,
                         data_root=cfg.data_root,
                         fullframe_size=[cfg.number_r_grid , cfg.number_z_grid],
                         fpds=cfg.frames_per_file, 
                         use_cache=use_cache,
                         verbose=True)
else:
    reader = H5Reader(str(os.path.join(cfg.data_path, 'data.h5')), use_cache=use_cache)
    reader.verbose = True

In [ ]:
# get row number
row_number = cfg.get_row_by_radius(radius)

# get data
data = []

for probe in cfg.probes:
    frame = cfg.get_frame_number_by_timestamp(timestamp, probe.schedule)
    if (probe.type == 'frame') and (probe.specie == specie) and (probe.r_start == shape[0]) and (probe.z_start == shape[1]) and(probe.r_end == shape[2]) and(probe.z_end == shape[3]):
        if probe.component == 'density': data = reader.get_frame('density/{}'.format(probe.specie), shape, frame)[:, row_number]

# try bigger frames, if autoselect enabled
if len(data) == 0 and autoselect:
    for probe in cfg.probes:
        probe_shape = [probe.r_start, probe.z_start, probe.r_end, probe.z_end]
        frame = cfg.get_frame_number_by_timestamp(timestamp, probe.schedule)
        if (probe.type == 'frame') and (probe.specie == specie) and (probe_shape[0] <= shape[0]) and (probe_shape[1] <= shape[1]) and(probe_shape[2] >= shape[2]) and(probe_shape[3] >= shape[3]):
            if probe.component == 'density' and len(data) == 0: data = reader.get_frame('density/{}'.format(probe.specie), probe_shape, frame)[shape[0]:shape[2], shape[1]:shape[3]][:, row_number]

In [ ]:
# get data size
data_len = np.linspace(shape[0], shape[2], len(data))

# define plot builder
plot = PlotBuilder(0, 0, # let the system detects sizes automatically
                   fig_color=cfg.figure_color, 
                   fig_width=cfg.figure_width,
                   fig_height=cfg.figure_height, 
                   fig_dpi=cfg.figure_dpi,
                   font_family=cfg.figure_font_family,
                   font_name=cfg.figure_font_name,
                   font_size=cfg.figure_font_size,
                   tickbox=True, grid=show_grid, is_invert_y_axe=False,
                   aspect='auto', guess_number_ticks=20,
                   # number_x_ticks=10, number_y_ticks=10
                   x_ticklabel_end=1e-9, y_ticklabel_end=1e-9
                  )

# add subplots
plot_d = plot.add_subplot_cartesian_2d(t_plot_name, 111, x_axe_label=x_axis_label, y_axe_label=y_axis_label)

# set y-limits
if ylim is not None:
    plot_d.set_ylim(ylim)

In [ ]:
plot.get_figure().suptitle("Time: {:.2e} s".format(timestamp), x=.85, y=.95)

# add data
plot_d.plot(data_len, data)

plot.show()